**Общий пайплайн решения такой:**

0) Скачиваем данные, смотрим на них глазами

1) Делаем предобработку данных

2) Делаем обратку NER

3) Обрабатываем особенности

4) Постобработка

5) Сохраняем результат

# Этап 0 : Скачиваем данные

Я работаю в колабе, поэтому предварительно загрузил csv файл на гугл диск и буду его вытаскивать оттуда.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = "/content/drive/MyDrive/analyzer_analyzer_urls.csv"

import pandas as pd

df = pd.read_csv(file_path)

Смотрим на форму данных и на то, что они из себя представляют

In [3]:
print(df.shape)
print(df.columns.to_list())

(9999, 2)
['http://0-9.ru', '<html><head>\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>\n        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">\n        <meta name="application-name" content="nic.ru">\n        <meta name="robots" content="noyaca">\n        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">\n        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">\n        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">\n        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">\n        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">\n        

In [4]:
print(df.columns.to_list()[1])

<html><head>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>
        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">
        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">
        <meta name="application-name" content="nic.ru">
        <meta name="robots" content="noyaca">
        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">
        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">
        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">
        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">
        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">
        <meta property="og:description" id="descr

Значит данные представляют из себя 2 столбца - "link" - "html". Переименую их для удобной работы

In [5]:
link1, html1 = df.columns.to_list()

In [6]:
print(link1, html1)

http://0-9.ru <html><head>
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>
        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">
        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">
        <meta name="application-name" content="nic.ru">
        <meta name="robots" content="noyaca">
        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">
        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">
        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">
        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">
        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">
        <meta property="og:descript

Обратим внимание, что названия столбцов отдельно не выделены, а на их месте стоит первый обект. Чтобы не создавать копию df(имею ограниченные ресурсы), я просто отдельно сохраню эту строчку и для удобства переименую столбцы.

In [7]:
df.columns = ['link', 'html']

In [8]:
df.columns.to_list()

['link', 'html']

In [9]:
df['link']

,link
0,http://0009.ru
1,http://001k.ru
2,http://003ms.ru
3,http://003rt.ru
4,http://004.ru
...,...
9994,http://1hobby.ru
9995,http://1hod.ru
9996,http://1hop.ru
9997,http://1hostels.ru


Отлично, данные я загрузил, на них глазами посмотрел. Дальше будем их предобрабатывать.

## Этап 1 : Предобработка данных

Я понимаю, что html-разметка несет в себе только служебную информацию о странице и будет мешать модели искать сущности, поэтому необходимо очистить данные от нее.

Поэтому сейчас цель : извлечь чистый текст, сохранив семантику и контекст сущностей.



Удаляем мусорные теги

In [10]:
from bs4 import BeautifulSoup, Comment
import re

def clean_html(html):
    soup = BeautifulSoup(html, 'lxml')

    blacklist = ['script', 'style', 'meta', 'link', 'footer',
                 'nav', 'aside', 'iframe', 'svg', 'noscript']
    for tag in soup(blacklist):
        tag.decompose()

    for tag in soup.find_all(True):
        if 'style' in tag.attrs:
            del tag['style']

    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    for tag in soup.find_all(re.compile(r'^.*$')):
        if len(tag.get_text(strip=True)) == 0:
            tag.decompose()

    return soup

Извлекаем текст с контекстом

In [11]:
def extract_structured_text(soup):
    structure_rules = {
        'h1': '\n# ',    # Заголовки как маркеры
        'h2': '\n## ',
        'p': '\n',
        'li': '\n * '    # Элементы списка
    }

    text = []
    for tag in soup.find_all(list(structure_rules.keys()) + ['div', 'span']):
        if tag.name in structure_rules:
            prefix = structure_rules[tag.name]
        else:
            prefix = ' '

        content = tag.get_text(' ', strip=True)
        if content:
            text.append(f"{prefix}{content}")

    return ''.join(text)

Обрабатываем особенные кейсы типа : текст разбит тегами или реклама маскируется через контент

In [12]:
def merge_split_words(soup):
    for tag in soup.find_all(['a', 'span']):
        if tag.parent and tag.parent.name == 'p':
            tag.unwrap()  # Удаляем тег, оставляя текст
    return soup

In [13]:
def remove_ads(soup):
    ad_keywords = ['ad', 'banner', 'advert']

    for tag in soup.find_all(True):  # True = все теги
        classes = tag.attrs.get('class', []) if tag.attrs else []

        if isinstance(classes, list):  # Убедимся, что classes - это список
            class_str = ' '.join(classes).lower()
        else:  # На случай, если classes - строка (редкие случаи)
            class_str = classes.lower() if classes else ''

        if any(keyword in class_str for keyword in ad_keywords):
            tag.decompose()

        if tag.attrs and 'data-ad' in tag.attrs:
            tag.decompose()

    return soup

Посмотрим, что получается в итоге на примере

In [14]:
cl_soup = clean_html(html1)
cl_soup = remove_ads(cl_soup)
cl_soup = merge_split_words(cl_soup)
text1 = extract_structured_text(cl_soup)

In [15]:
html1

'<html><head>\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n        <title>Домен продается. Купить в магазине доменов RU-CENTER</title>\n        <meta name="description" content="Узнать подробнее о домене в магазине доменов RU-CENTER.">\n        <meta name="keywords" content="домен, регистрация доменов, РФ, RU, COM, аукцион доменов, хостинг, почта, освобождающиеся домены">\n        <meta name="application-name" content="nic.ru">\n        <meta name="robots" content="noyaca">\n        <meta name="msapplication-square70x70logo" content="/favicon_70x70.png">\n        <meta name="msapplication-square150x150logo" content="/favicon_150x150.png">\n        <meta name="msapplication-wide310x150logo" content="/favicon_310x150.png">\n        <meta name="msapplication-square310x310logo" content="/favicon_310x310.png">\n        <meta property="og:title" id="title" content="Домен продается. Купить в магазине доменов RU-CENTER">\n        <meta property="og:descripti

In [16]:
text1

' Купить в RU-CENTER Другие домены в магазине доменов Купить в RU-CENTER Другие домены в магазине доменов Купить в RU-CENTER Другие домены в магазине доменов Купить в RU-CENTER Купить в RU-CENTER Купить в RU-CENTER Купить в RU-CENTER Другие домены в магазине доменов Другие домены в магазине доменов Другие домены в магазине доменов Другие домены в магазине доменов'

Неплохо получилось.

In [17]:
def clean_pipeline(html):
  cl_soup = clean_html(html)
  cl_soup = remove_ads(cl_soup)
  cl_soup = merge_split_words(cl_soup)
  text = extract_structured_text(cl_soup)
  return text

## Этап 2 : Обработка NER

Тут надо выбрать дальнейший путь - какую модель использовать.



Вариант 1 : spacy. Она быстрая, но дает среднее качество

Вариант 2 : BERT/RoBERTa. Медленная, но дает качество лучше

Вариант 3 : зафайнтюненый BERT. Медленая, но дает лучшее качество. Этот вариант недоступен, поскольку нет обучающей выборки, но потенциально, если были бы, то почему бы и нет.

Короче после многих экспериментов я пришел к выводу, что буду использовать spacy для русского языка для поиска контактов, а почты и номера с помощью регулярок. На встрече можем лучше обсудить, почему я пошел по первому пути - причины есть, но они инженерные.

In [18]:
!pip install spacy
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 814.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [19]:
import spacy

# Загрузка модели
nlp = spacy.load("ru_core_news_lg")
nlp.max_length = 3000000

In [20]:
def extract_contacts(text):

    contacts = {
        'phones': set(),
        'emails': set(),
        'addresses': set(),
        'persons': set(),
        'organizations': set()
    }
    doc = nlp(text)
      # Регулярные выражения для основных типов контактов
    phone_regex = r'(?:\+7|8)[\s\-]?\(?\d{3}\)?[\s\-]?\d{3}[\s\-]?\d{2}[\s\-]?\d{2}'
    email_regex = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
    contacts['phones'].update(re.findall(phone_regex, text))
    contacts['emails'].update(re.findall(email_regex, text))

    #print(doc)
    # Анализ NER-результатов
    for ent in doc.ents:
        if ent.label_ == 'PER':
            contacts['persons'].add(ent.text)
        elif ent.label_ == 'LOC':
            contacts['addresses'].add(ent.text)
        elif ent.label_ == 'ORG':
            contacts['organizations'].add(ent.text)
    return contacts

In [21]:
extract_contacts('Александр идет в МФТИ в Москве +79044691080 чтобы написать в Тюмень по почте mail@mail.ru')
print()

In [22]:
# Пример текста
text = "Компания Яндекс основана в 1997 году Аркадием Воложем и Ильей Сегаловичем в Москве."

# Обработка текста
doc = nlp(text)

# Извлечение сущностей
for ent in doc.ents:
    print(f"Текст: {ent.text}, Тип: {ent.label_}, Позиция: {ent.start_char}-{ent.end_char}")

Текст: Яндекс, Тип: ORG, Позиция: 9-15
Текст: Аркадием Воложем, Тип: PER, Позиция: 37-53
Текст: Ильей Сегаловичем, Тип: PER, Позиция: 56-73
Текст: Москве, Тип: LOC, Позиция: 76-82


In [23]:
data_df = pd.DataFrame(columns = ['page_id', 'url', 'phones', 'emails', 'addresses', 'persons', 'organizations'])

In [24]:
contacts = extract_contacts(text1)
new_row = {
              'page_id': 0,
              'url': df['link'][0],
              'phones': contacts['phones'],
              'emails': contacts['emails'],
              'addresses': contacts['addresses'],
              'persons': contacts['persons'],
              'orsganizations' : contacts['organizations']
          }

data_df = pd.concat([data_df, pd.DataFrame([new_row])], ignore_index=True)

In [25]:
data_df

,page_id,url,phones,emails,addresses,persons,organizations,orsganizations
0,0,http://0009.ru,{},{},{Купить},{Другие},NaN,{}


In [26]:
from tqdm import tqdm
import pandas as pd

# Инициализация DataFrame с нужными колонками
data_df = pd.DataFrame(columns=['page_id', 'url', 'phones', 'emails', 'addresses', 'persons', 'organizations'])

# Обработка с прогресс-баром
for i in tqdm(range(df.shape[0]), desc="Обработка страниц", leave=False):
    doc = clean_pipeline(df['html'][i])
    contacts = extract_contacts(doc)

    # Добавляем данные через словарь
    data_df.loc[len(data_df)] = {
        'page_id': i + 1,
        'url': df['link'][i],
        'phones': contacts['phones'],
        'emails': contacts['emails'],
        'addresses': contacts['addresses'],
        'persons': contacts['persons'],
        'organizations': contacts['organizations']
    }

In [27]:
data_df.to_csv('extracted_entities.csv', index=False, encoding='utf-8-sig')

In [28]:
from google.colab import drive
import shutil

# Монтируем Google Drive
drive.mount('/content/drive')

# Копируем файл в нужную папку на Drive
shutil.copy('/content/extracted_entities.csv', '/content/drive/MyDrive/')

print("Файл успешно сохранён в Google Drive!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Файл успешно сохранён в Google Drive!


In [29]:
# from transformers import pipeline
# import torch

# ner_pipeline = pipeline(
#     "ner",
#     model="Gherman/bert-base-NER-Russian", # Короче это просто копия сберовской модели
#     device=0 if torch.cuda.is_available() else -1,
#     aggregation_strategy="simple"
# )

In [30]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("Gherman/bert-base-NER-Russian")

In [31]:
# def process_long_text(text, window_size=400, stride=50):
#     tokens = tokenizer.tokenize(text)
#     results = []

#     for i in range(0, len(tokens), stride):
#         window = tokens[i:i+window_size]
#         window_text = tokenizer.convert_tokens_to_string(window)
#         results.extend(ner_pipeline(window_text))

#     return results

In [32]:
# process_long_text('Касюк Вадим поехал в Москву чтобы позвонить +79044691080 на фабрику')

Честно говоря, я уже устал пробовать разные модельки, поэтому просто вернусь к spacy и все